<h1>Empleamos el ONI</h1>

In [1]:
#import geopandas as gpd
import pandas as pd
import xarray as xr
import numpy as np
#from tools import search_oni
import skimage
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
#from tools import filt_anom2

In [2]:
data_oni = pd.read_csv('E:\Proyecto de Grado\Data\ONI_NOAA.txt', sep="\s+")
data_oni['TIME'] = pd.to_datetime(data_oni['TIME'], format = '%Y-%m')

In [3]:
data_month = pd.read_csv('E:\Proyecto de Grado\Data\ONI_NOAA - copia.txt', sep="\s+")

In [4]:
"""años_niña = search_oni('negativo',data_oni)
años_niño = search_oni('positivo',data_oni)
años_neutro = search_oni('neutro',data_oni)"""

"años_niña = search_oni('negativo',data_oni)\naños_niño = search_oni('positivo',data_oni)\naños_neutro = search_oni('neutro',data_oni)"

Extraemos los datos del netCDF

In [5]:
ds_currents = xr.open_dataset('E:\Proyecto de Grado\Data\cmems_mod_glo_phy_my_0.083_P1M-m_1695317394750.nc')
ds_currents

<xarray.Dataset>
Dimensions:    (depth: 5, latitude: 190, time: 336, longitude: 363)
Coordinates:
  * depth      (depth) float32 0.494 1.541 2.646 3.819 5.078
  * latitude   (latitude) float32 7.667 7.75 7.833 7.917 ... 23.25 23.33 23.42
  * time       (time) datetime64[ns] 1993-01-16T12:00:00 ... 2020-12-16T12:00:00
  * longitude  (longitude) float32 -89.17 -89.08 -89.0 ... -59.17 -59.08 -59.0
Data variables:
    vo         (time, depth, latitude, longitude) float32 ...
    thetao     (time, depth, latitude, longitude) float32 ...
    uo         (time, depth, latitude, longitude) float32 ...
Attributes: (12/17)
    title:                         Monthly mean fields for product GLOBAL_REA...
    references:                    http://marine.copernicus.eu
    credit:                        E.U. Copernicus Marine Service Information...
    licence:                       http://marine.copernicus.eu/services-portf...
    contact:                       servicedesk.cmems@mercator-ocean.eu
    producer:                      CMEMS - Global Monitoring and Forecasting ...
    ...                            ...
    source:                        MERCATOR GLORYS12V1
    product_user_manual:           http://marine.copernicus.eu/documents/PUM/...
    quality_information_document:  http://marine.copernicus.eu/documents/QUID...
    _CoordSysBuilder:              ucar.nc2.dataset.conv.CF1Convention
    comment:                       
    history:                       Data extracted from dataset http://localho...

In [6]:
data_oni['category'] = data_oni['ANOM'].apply(lambda x: 'Niño' if x >= 0.5 else 'Neutro' if x > -0.5 else 'Niña')

In [7]:
ds_currents = ds_currents.expand_dims(ONI = data_oni['category']).assign_coords(ONI = data_oni['category'])


In [8]:
ds_currents = ds_currents.expand_dims(month = data_month['MON']).assign_coords(month = data_month['MON'])

In [9]:
ds_currents

<xarray.Dataset>
Dimensions:    (dim_0: 336, ONI: 336, depth: 5, latitude: 190, month: 336,
                time: 336, longitude: 363)
Coordinates:
  * month      (dim_0) int64 1 2 3 4 5 6 7 8 9 10 11 ... 3 4 5 6 7 8 9 10 11 12
  * ONI        (ONI) object 'Neutro' 'Neutro' 'Neutro' ... 'Niña' 'Niña' 'Niña'
  * depth      (depth) float32 0.494 1.541 2.646 3.819 5.078
  * latitude   (latitude) float32 7.667 7.75 7.833 7.917 ... 23.25 23.33 23.42
  * time       (time) datetime64[ns] 1993-01-16T12:00:00 ... 2020-12-16T12:00:00
  * longitude  (longitude) float32 -89.17 -89.08 -89.0 ... -59.17 -59.08 -59.0
  * dim_0      (dim_0) int64 0 1 2 3 4 5 6 7 ... 328 329 330 331 332 333 334 335
Data variables:
    vo         (month, ONI, time, depth, latitude, longitude) float32 0.2167 ...
    thetao     (month, ONI, time, depth, latitude, longitude) float32 27.22 ....
    uo         (month, ONI, time, depth, latitude, longitude) float32 0.0647 ...
Attributes: (12/17)
    title:                         Monthly mean fields for product GLOBAL_REA...
    references:                    http://marine.copernicus.eu
    credit:                        E.U. Copernicus Marine Service Information...
    licence:                       http://marine.copernicus.eu/services-portf...
    contact:                       servicedesk.cmems@mercator-ocean.eu
    producer:                      CMEMS - Global Monitoring and Forecasting ...
    ...                            ...
    source:                        MERCATOR GLORYS12V1
    product_user_manual:           http://marine.copernicus.eu/documents/PUM/...
    quality_information_document:  http://marine.copernicus.eu/documents/QUID...
    _CoordSysBuilder:              ucar.nc2.dataset.conv.CF1Convention
    comment:                       
    history:                       Data extracted from dataset http://localho...

In [17]:
ds_NIÑO = ds_currents.where(ds_currents['ONI'] == 'Niño')

ValueError: cannot re-index or align objects with conflicting indexes found for the following dimensions: 'dim_0' (2 conflicting indexes)
Conflicting indexes may occur when
- they relate to different sets of coordinate and/or dimension names
- they don't have the same type
- they may be used to reindex data along common dimensions

In [24]:
ds_ONI = ds_currents.groupby(["ONI"],["month"]).mean()

TypeError: `group` must be an xarray.DataArray or the name of an xarray variable or dimension. Received ['ONI'] instead.

In [12]:
lat = ds_enero_niño['latitude'].to_numpy()
lon = ds_enero_niño['longitude'].to_numpy()
V = ds_enero_niño["vo"].mean(dim=["month","depth"]).to_numpy()
U = ds_enero["uo"].mean(dim=["month","depth"]).to_numpy()

X, Y = np.meshgrid(lon, lat)

x = skimage.measure.block_reduce(X, (7, 7), np.nanmean)
y = skimage.measure.block_reduce(Y, (7, 7), np.nanmean)
u = skimage.measure.block_reduce(U, (7, 7), np.nanmean)
v = skimage.measure.block_reduce(V, (7, 7), np.nanmean)
# Crea una figura y ejes con la proyección de latitud y longitud
fig, ax = plt.subplots(1, figsize=(8, 6), subplot_kw={'projection': ccrs.PlateCarree()})
loc = ticker.MultipleLocator(4)

# Agrega el mapa base (costas, límites de países, etc.)
ax.add_feature(cf.COASTLINE)
ax.add_feature(cf.BORDERS)

# Crea el gráfico de vectores
Q = ax.quiver(x, y, u, v, np.sqrt(u**2 + v**2), scale=10, cmap= 'jet')

# Agrega una barra de color asociada a la magnitud de los vectores
plt.colorbar(Q, label=r"Velocity [$\frac{m}{s}$]", shrink = 0.5)

# Establece los límites del mapa y etiquetas de los ejes
ax.set(ylim=(lat.min(), lat.max()), xlim=(lon.min(), lon.max()))

# Agrega un marco naval con las coordenadas en rojo y con líneas discontinuas
ax.gridlines(draw_labels={"bottom": "x", "left": "y"}, xlocs=loc, ylocs=loc)

# Muestra el mapa
plt.title("Velocidades de Corrientes superficiales de Enero (NIÑO)")
plt.show()

'lat = ds_enero[\'latitude\'].to_numpy()\nlon = ds_enero[\'longitude\'].to_numpy()\nV = ds_enero["vo"].mean(dim=["month","depth"]).to_numpy()\nU = ds_enero["uo"].mean(dim=["month","depth"]).to_numpy()\n\nX, Y = np.meshgrid(lon, lat)\n\nx = skimage.measure.block_reduce(X, (7, 7), np.nanmean)\ny = skimage.measure.block_reduce(Y, (7, 7), np.nanmean)\nu = skimage.measure.block_reduce(U, (7, 7), np.nanmean)\nv = skimage.measure.block_reduce(V, (7, 7), np.nanmean)\n# Crea una figura y ejes con la proyección de latitud y longitud\nfig, ax = plt.subplots(1, figsize=(8, 6), subplot_kw={\'projection\': ccrs.PlateCarree()})\nloc = ticker.MultipleLocator(4)\n\n# Agrega el mapa base (costas, límites de países, etc.)\nax.add_feature(cf.COASTLINE)\nax.add_feature(cf.BORDERS)\n\n# Crea el gráfico de vectores\nQ = ax.quiver(x, y, u, v, np.sqrt(u**2 + v**2), scale=10, cmap= \'jet\')\n\n# Agrega una barra de color asociada a la magnitud de los vectores\nplt.colorbar(Q, label=r"Velocity [$\x0crac{m}{s}$]"